# CryptoDataPy Demo

**CryptoDataPy facilitates the pre-processing of data for the analysis of digital assets**. This includes **market** (e.g. tick, trade and bar data for cryptoassets and traditional assets), **on-chain** (e.g. network usage and supply data) and **off-chain** data (e.g. news, social media metrics, macroeconomic data, etc).


In this notebook, we will walk through how to use **CryptoDataPy** to query various types of data from multiple sources and get it ready for the data analysis of digital assets. 

Key features of the library include:

- **`Data Catalog`**:
    - *Data sources*: get information on the available data sources. 
    - *Tickers*: get metadata on available tickers. 
    - *Fields*: get metadata on available data fields. 
    
    
- **`Data Requests`**:
    - *Data requests*: specify the query parameters for the data you want.
    - *Get data*: converts data request parameters to the data source's format, retrieves the data and wrangles it into [tidy data](https://www.jstatsoft.org/article/view/v059i10) format.


- **`Data Vendors`**:
    - *Data credentials*: requesting and storing your api keys.
    - *Metadata*: get metadata from data source.


- **`Data Cleaning`**:
    - *Outlier detection*: detecting outliers in raw data.
    - *Repair*: dealing with missing values or outliers in the data. 
    - *Filters*: filtering 'bad data' based on specific criteria (e.g. assets with low liquidity, stablecoins, etc).
    - *Cleaning*: creating a data cleaning pipeline to get data ready for analysis.


# Data Catalog

The `DataCatalog` allow us to explore what data is available and understand it better.

To access the data catalog, instantiate a DataCatalog object:

In [19]:
from cryptodatapy.util.datacatalog import DataCatalog 
dc = DataCatalog()

### Data Sources

Available `data sources` 


In [22]:
dc.data_sources

{'ccxt': 'https://github.com/ccxt/ccxt',
 'cryptocompare': 'https://min-api.cryptocompare.com/documentation',
 'coinmetrics': 'https://docs.coinmetrics.io/info/markets',
 'glassnode': 'https://glassnode.com/',
 'tiingo': 'https://api.tiingo.com/products/crypto-api',
 'av-daily': 'https://www.alphavantage.co/documentation/',
 'av-forex-daily': 'https://www.alphavantage.co/documentation/',
 'yahoo': 'https://finance.yahoo.com/',
 'investpy': 'https://investpy.readthedocs.io/',
 'dbnomics': 'https://db.nomics.world/providers',
 'fred': 'https://fred.stlouisfed.org/'}

### Tickers

Available tickers can be accessed with the `get_tickers_metadata` method:

In [20]:
dc.get_tickers_metadata()

,name,description,url,country_id_2,country_id_3,country_name,agg,category,subcategory,mkt_type,frequency,tenor,unit,quote_ccy,investpy_id,tiingo_id,dbnomics_id,fred_id,alfred_id,philly_fed_id
ticker,,,,,,,,,,,,,,,,,,,,
ARS,Argentine Peso,"Argentine Peso vs. quote currency, spot exchan...",NaN,AR,ARG,Argentina,EM,fx,spot rate,spot,tick,NaN,foreign currency per unit of domestic currrency,NaN,ARS,ARS,NaN,NaN,NaN,NaN
AUD,Australian Dollar,"Australian Dollar vs. quote currency, spot exc...",NaN,AU,AUS,Australia,DM,fx,spot rate,spot,tick,NaN,foreign currency per unit of domestic currrency,NaN,AUD,AUD,NaN,NaN,NaN,NaN
EUR,Euro,"Euro vs. quote currency, spot exchange rate",NaN,AT,AUT,Austria,DM,fx,spot rate,spot,tick,NaN,foreign currency per unit of domestic currrency,NaN,EUR,EUR,NaN,NaN,NaN,NaN
EUR,Euro,"Euro vs. quote currency, spot exchange rate",NaN,BE,BEL,Belgium,DM,fx,spot rate,spot,tick,NaN,foreign currency per unit of domestic currrency,NaN,EUR,EUR,NaN,NaN,NaN,NaN
BDT,Bangladeshi Taka,"Bangladeshi Taka vs. quote currency, spot exch...",NaN,BD,BGD,Bangladesh,NaN,fx,spot rate,spot,tick,NaN,foreign currency per unit of domestic currrency,NaN,BDT,BDT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JCHF,Jarvis Synthetic,NaN,NaN,WL,WLD,World,WL,crypto,stablecoin,spot,tick,NaN,fiat currency per unit of stablecoin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
XUSD,xDollar Stablecoin,NaN,NaN,WL,WLD,World,WL,crypto,stablecoin,spot,tick,NaN,fiat currency per unit of stablecoin,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IRON,Iron BSC,NaN,NaN,WL,WLD,World,WL,crypto,stablecoin,spot,tick,NaN,fiat currency per unit of stablecoin,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Data Vendors

Pulling data from vendors generally requies either becoming familiar with their web APIs (by reviewing their documentation) or installing an API wrapper. 

With `cryptodatapy` you can interact with any supported data vendor's web API and access its data using our intuitive interface. Below, we'll show to access data from various data vendors. 

## Data Credentials

Most data vendors will require an API key to access their data so we must first request and store those API keys. 

We recommend storing API keys, along with all other data credentials, in the DataCredentials dataclass. It acts as a data container which stores all relevant data constants for **CryptoDataPy**. 

In [1]:
# import DataCredentials to set CryptoCompare API key
from cryptodatapy.util.datacredentials import DataCredentials, set_credential
# uncomment and run line below
# set_credential('cryptocompare_api_key', {'insert your api key here'}) # this only needs to be run once

Once we have obtained and set our API keys, we can import the data vendor's class and initialize an object. 
<br>
If your API key is not set, it will be returned as None from DataCredentials. You should therefore pass it as a parameter in the data vendor's object, as shows below with CryptoCompare.

In [2]:
# import CryptoCompare
from cryptodatapy.extract.data_vendors.cryptocompare import CryptoCompare
# initialize CryptoCompare
cc = CryptoCompare() # if api key set using set_credential in datacredentials.py
# else insert your cryptocompare api key, uncomment and run the code below
# cc = CryptoCompare(api_key={your cryptocompare api key})

Now, we can explore some of the available data...

## Metadata

`cryptodatapy`'s interface makes it easy to explore any data vendor's available data. Let's explore CryptoCompare as an example. 

### Source Type

To get the data source type

In [3]:
cc.source_type

'data_vendor'

### Categories

Categories are either assets classes or types of time series data (e.g. macro, alt data, etc.).

To get a data vendor's available categories

In [4]:
cc.categories

['crypto']

### Assets

To get a list of available assets

In [5]:
cc.assets

['42',
 '300',
 '365',
 '404',
 '433',
 '611',
 '808',
 '888',
 '1337',
 '1717',
 '2015',
 'BSEND',
 'DESTINY',
 'BIT16',
 'PDC',
 'CMTC',
 'CHESSCOIN',
 'SPACECOIN',
 'REE',
 'LQD',
 'MARV',
 'VEC2',
 'OMNI',
 'GSY',
 'TRTK',
 'LIR',
 'MMNXT',
 'SCRPT',
 'LBC',
 'SPCIE',
 'SBD',
 'CJ',
 'PUT',
 'KRAK',
 'DLISK',
 'IBANK',
 'STRAX',
 'VOYA',
 'ENTER',
 'WRLGC',
 'BM',
 'FRWC',
 'PSILOC',
 'XTREME',
 'RUST',
 'NZC',
 'SNGLS',
 'XAUR',
 'BFX',
 'UNIQ',
 'CRX',
 'DCT',
 'XPOKE',
 'MUDRA',
 'WARP',
 'CNMT',
 'PIZZACOIN',
 'LUTETIUM',
 'HEAT',
 'ICN',
 'EXB',
 'WINGS',
 'CDEX',
 'RBIT',
 'DCS.',
 'KMD',
 'GB',
 'NEO',
 'ANONCOIN',
 'SYNX',
 'EDC',
 'JWL',
 'WAY',
 'TAB',
 'TRIG',
 'BITCNY',
 'BITUSD',
 'ATMC',
 'STO',
 'CARTERCOIN',
 'TOT',
 'BTD',
 'BOTS',
 'MDC',
 'FTP',
 'ZET2',
 'CVNC',
 'KRB',
 'TELL',
 'ENE',
 'TDFB',
 'BLOCKPAY',
 'BXT',
 'ZYD',
 'MUSTANGC',
 'GOON',
 'VLT',
 'ZNE',
 'DCK',
 'COVAL',
 'DGDC',
 'TODAY',
 'VRM',
 'ROOT',
 '1ST',
 'GPL',
 'DOPE',
 'B3',
 'FX',
 'PIO',
 

For more granular asset info 

In [6]:
cc.get_assets_info()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,Algorithm,ProofType,SortOrder,Sponsored,Taxonomy,Rating,IsTrading
ticker,,,,,,,,,,,,,,,,,
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,Scrypt,PoW/PoS,34,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True
300,749869,#overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,N/A,N/A,2212,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True
365,33639,#overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,X11,PoW/PoS,916,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,Scrypt,PoW/PoS,602,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True
433,926547,#overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,N/A,N/A,3505,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
EVIL,22272,/coins/evil/overview,/media/351080/evil.png,1467125887,EVIL,EVIL,EvilCoin,EvilCoin (EVIL),"EvilCoin was created to embrace, reward and in...",N/A,X11,PoW/PoS,653,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True
OMA,22280,/coins/oma/overview,/media/35651316/omega300x300.png,1467129110,OMA,OMA,OmegaCoin,OmegaCoin (OMA),OmegaCoin is a PoW/PoS hybrid cryptocurrency t...,N/A,X15,PoW/PoS,654,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True
MUE,22325,/coins/mue/overview,/media/35651317/monetaryunit300x300.png,1467194997,MUE,MUE,MonetaryUnit,MonetaryUnit (MUE),MonetaryUnit is a cryptocurrency based on the ...,N/A,Quark,PoW,655,False,"{'Access': '', 'FCA': '', 'FINMA': '', 'Indust...","{'Weiss': {'Rating': '', 'TechnologyAdoptionRa...",True


To get top market cap assets

In [7]:
cc.get_top_market_cap_assets()

['BTC',
 'ETH',
 'USDT',
 'USDC',
 'BNB',
 'XRP',
 'SOL',
 'BUSD',
 'ADA',
 'DOGE',
 'DOT',
 'FTT',
 'AVAX',
 'TRX',
 'DAI',
 'LINK',
 'SHIB',
 'GMT',
 'XLM',
 'UNI',
 'LEO',
 'WBTC',
 'MATIC',
 'APE',
 'AXS',
 'SAND',
 'LTC',
 'CRO',
 'OKB',
 'ATOM',
 'ICP',
 'KLAY',
 'NEAR',
 'ALGO',
 'XMR',
 'LUNA',
 'QNT',
 'GALA',
 'BCH',
 'IMX',
 'VET',
 'MANA',
 'DYDX',
 'CRV',
 'FLOW',
 'TITAN',
 'XTZ',
 'HBAR',
 'KCS',
 'EGLD',
 'FIL',
 'PCI',
 'THETA',
 'GNO',
 'HNT',
 'XCH',
 'RUNE',
 'BSV',
 'EOS',
 'GRT',
 'AAVE',
 'ENS',
 'XDC',
 'HT',
 '1INCH',
 'MKR',
 'CELO',
 'AMP',
 'CHZ',
 'USDP',
 'NEO',
 'LUNC',
 'ILV',
 'AR',
 'DFI',
 'XEC',
 'XRD',
 'MIOTA',
 'USDN',
 'TWT',
 'SNX',
 'VELO',
 'ZEC',
 'MC',
 'RSR',
 'GLMR',
 'FTM',
 'YGG',
 'BAT',
 'NU',
 'ZIL',
 'GT',
 'CEL',
 'MINA',
 'NEXO',
 'MX',
 'PAXG']

### Indexes

To get a list of available indexes

In [8]:
cc.indexes

['MVDA', 'MVDALC', 'MVDAMC', 'MVDASC', 'BTCA', 'BVIN', 'BTCB', 'ETHB']

For more granular index info

In [9]:
cc.get_indexes_info()

,name,description,quote_currency,index_market_name,index_market_underlying_name,day_close_in_seconds
ticker,,,,,,
MVDA,MVIS CryptoCompare Digital Assets 100,,USD,CCMVDA,CCMVDA,61200
MVDALC,MVIS CryptoCompare Digital Assets 100 Large-Cap,,USD,CCMVDA,CCMVDA,61200
MVDAMC,MVIS CryptoCompare Digital Assets 100 Mid-Cap,,USD,CCMVDA,CCMVDA,61200
MVDASC,MVIS CryptoCompare Digital Assets 100 Small-Cap,,USD,CCMVDA,CCMVDA,61200
BTCA,FICAS Actively Managed ETP,,CHF,CCNAV,CCMVDA,50400
BVIN,CryptoCompare Bitcoin Volatility Index,,USD,CCMVDA_VIX,CCMVDA_VIX,36000
BTCB,FICAS Bitcoin Fund,,CHF,CCNAV,CCMVDA,61200
ETHB,FICAS Ethereum Fund,,CHF,CCNAV,CCMVDA,61200


### Markets
To get a list of availalble markets (asset:quote_ccy)

In [10]:
cc.markets

['365BTC',
 '365RUR',
 '365ETH',
 '365DOGE',
 '365USD',
 '365WAVES',
 '404XRP',
 '404USD',
 '404WAVES',
 '404RUR',
 '404ETH',
 '404DOGE',
 '888BTC',
 '2015BTC',
 'ARBTUSDT',
 'IGGBTC',
 'IGGTRX',
 'TTUUSDT',
 'TTUETH',
 'TTUBTC',
 'BTZCUSDT',
 'BTZCBTC',
 'LOGBTC',
 'ZECCKUSD',
 'ZECUSDC',
 'ZECETH',
 'ZECTUSD',
 'ZECBNB',
 'ZECBUSD',
 'ZECUSDP',
 'ZECUSDT',
 'ZECBTC',
 'ZECUSD',
 'ZECBRL',
 'ZECGBP',
 'ZECAUD',
 'ZECCPEX',
 'ZECEOS',
 'ZECEURS',
 'ZECBCH',
 'ZECHUSD',
 'ZECKRW',
 'ZECCAD',
 'ZECJPY',
 'ZECEUR',
 'ZECKCS',
 'ZECUAH',
 'ZECXVG',
 'ZECOKB',
 'ZECXRP',
 'LTCCKUSD',
 'LTCUSDT',
 'LTCBTC',
 'LTCJPY',
 'LTCAUD',
 'LTCTRY',
 'LTCTHB',
 'LTCETH',
 'LTCBNB',
 'LTCUSDC',
 'LTCEUR',
 'LTCTUSD',
 'LTCBUSD',
 'LTCNGN',
 'LTCUAH',
 'LTCGBP',
 'LTCBRL',
 'LTCILS',
 'LTCPLN',
 'LTCUSD',
 'LTCKRW',
 'LTCMXN',
 'LTCPEN',
 'LTCCLP',
 'LTCARS',
 'LTCCOP',
 'LTCBCH',
 'LTCCPEX',
 'LTCCZK',
 'LTCEURS',
 'LTCCNYT',
 'LTCRUB',
 'LTCDASH',
 'LTCDAI',
 'LTCSAI',
 'LTCEOS',
 'LTCHT',
 'LTCHUSD',

For more granular markets info

In [11]:
cc.get_markets_info()

{'365': {'BTC': {'exchanges': {'Yobit': {'isActive': True,
     'isTopTier': False}},
   'histo_minute_start_ts': 1495756800,
   'histo_minute_start': '2017-05-26',
   'histo_minute_end_ts': 1656806400,
   'histo_minute_end': '2022-07-03'},
  'RUR': {'exchanges': {'Yobit': {'isActive': True, 'isTopTier': False}},
   'histo_minute_start_ts': 1509408000,
   'histo_minute_start': '2017-10-31',
   'histo_minute_end_ts': 1656806400,
   'histo_minute_end': '2022-07-03'},
  'ETH': {'exchanges': {'Yobit': {'isActive': True, 'isTopTier': False}},
   'histo_minute_start_ts': 1510531200,
   'histo_minute_start': '2017-11-13',
   'histo_minute_end_ts': 1656806400,
   'histo_minute_end': '2022-07-03'},
  'DOGE': {'exchanges': {'Yobit': {'isActive': True, 'isTopTier': False}},
   'histo_minute_start_ts': 1509321600,
   'histo_minute_start': '2017-10-30',
   'histo_minute_end_ts': 1656806400,
   'histo_minute_end': '2022-07-03'},
  'USD': {'exchanges': {'Yobit': {'isActive': True, 'isTopTier': False}

### Market Types
 
To get a list of available market types

In [12]:
cc.market_types

['spot']

### Fields
To get a list of available fields

In [13]:
cc.fields

['open',
 'high',
 'low',
 'close',
 'volume',
 'zero_balance_addresses_all_time',
 'unique_addresses_all_time',
 'new_addresses',
 'active_addresses',
 'average_transaction_value',
 'block_height',
 'hashrate',
 'difficulty',
 'block_time',
 'block_size',
 'current_supply',
 'transaction_count',
 'transaction_count_all_time',
 'large_transaction_count',
 'comments',
 'posts',
 'followers',
 'points',
 'overview_page_views',
 'analysis_page_views',
 'markets_page_views',
 'charts_page_views',
 'trades_page_views',
 'forum_page_views',
 'influence_page_views',
 'total_page_views',
 'fb_likes',
 'fb_talking_about',
 'twitter_followers',
 'twitter_following',
 'twitter_lists',
 'twitter_favourites',
 'twitter_statuses',
 'reddit_subscribers',
 'reddit_active_users',
 'reddit_posts_per_hour',
 'reddit_posts_per_day',
 'reddit_comments_per_hour',
 'reddit_comments_per_day',
 'code_repo_stars',
 'code_repo_forks',
 'code_repo_subscribers',
 'code_repo_open_pull_issues',
 'code_repo_closed_pu

To get on chain asset info

In [14]:
cc.get_onchain_info()

,id,symbol,partner_symbol,data_available_from
ticker,,,,
0XBTC,877383,0XBTC,0XBTC,2018-02-07
1ST,28328,1ST,1ST,2016-12-09
1WO,925200,1WO,1WO,2017-11-30
AAC,925291,AAC,AAC,2017-12-29
ABCC,925864,ABCC,AT,2018-07-08
...,...,...,...,...
ZPR,925282,ZPR,ZPR,2018-05-17
ZRX,186277,ZRX,ZRX,2017-08-11
ZSC,341328,ZSC,ZSC,2017-07-09


### Frequencies

To get a list of available data (observations) frequencies

In [15]:
cc.frequencies

['1min', '1h', 'd']

### Exchanges
To get a list of available exchanges

In [16]:
cc.exchanges

['Bitstamp',
 'Bittrex',
 'BTCE',
 'Okcoin',
 'BTER',
 'Poloniex',
 'Kraken',
 'Bitfinex',
 'Cex.io',
 'Yacuna',
 'Coinbase',
 'LocalBitcoins',
 'itBit',
 'HitBTC',
 'BTC38',
 'Coinfloor',
 'Huobi US',
 'LakeBTC',
 'Bit2C',
 'Coinsetter',
 'CCEX',
 'Coinse',
 'MonetaGo',
 'Gatecoin',
 'Yunbi',
 'Gemini',
 'CCEDK',
 'Cryptopia',
 'Exmo',
 'Yobit',
 'BitBay',
 'Korbit',
 'BTCMarkets',
 'Coincheck',
 'Bisq',
 'Vaultoro',
 'Mercado Bitcoin',
 'Unocoin',
 'Bitso',
 'BTCXIndia',
 'Paymium',
 'The Rock Trading',
 'bitFlyer',
 'Quoine',
 'Liqui',
 'Luno',
 'CryptoX',
 'EtherDelta',
 'TuxExchange',
 'CHBTC',
 'bitFlyerFX',
 'Livecoin',
 'Tidex',
 'CoinCorner',
 'BitMarket',
 'Coinone',
 'Bleutrade',
 'ETHEXIndia',
 'Surbitcoin',
 'Bithumb Korea',
 'FoxBit',
 'Zaif',
 'Jubi',
 'ViaBTC',
 'NovaExchange',
 'Waves.Exchange',
 'Binance',
 'Lykke',
 'Remitano',
 'Coinroom',
 'Abucoins',
 'BX.in.th',
 'Huobi Global',
 'OKX',
 'Gate.io',
 'TRUSTdex',
 'EXX',
 'Kucoin',
 'BitGrail',
 'Bit-Z',
 'BitFlip'

For more granular exchanges info

In [17]:
cc.get_exchanges_info()

,Id,Url,LogoUrl,ItemType,CentralizationType,InternalName,GradePoints,Grade,GradePointsSplit,AffiliateURL,...,FullAddress,Fees,DepositMethods,WithdrawalMethods,Sponsored,Recommended,Rating,SortOrder,TOTALVOLUME24H,DISPLAYTOTALVOLUME24H
Name,,,,,,,,,,,,,,,,,,,,,
Bitstamp,2431,/exchanges/bitstamp/overview,/media/37748052/bitstamp.png,"[Cryptocurrency, Fiat]",Centralized,Bitstamp,76.68,AA,"{'Legal': '12.9', 'KYCAndTransactionRisk': '14...",https://links.bitstamp.net/AoaR5N,...,Bitstamp Ltd\n5 New Street Square\nLondon EC4A...,"See Bitstamp's <a href=""https://www.bitstamp.n...",Cryptocurrency\nFiat\n\nSee Bitstamp's <a href...,Cryptocurrency\nFiat\n\nSee Bitstamp's <a href...,False,False,"{'One': 108, 'Two': 25, 'Three': 16, 'Four': 2...",3,{'BTC': 10063.3450837149},{'BTC': 'Ƀ 10.06 K'}
Bittrex,2434,/exchanges/bittrex/overview,/media/37746429/bittrex.png,"[Cryptocurrency, Stable Coins, Tokens, Fiat]",Centralized,BitTrex,65.34,BB,"{'Legal': '9.1', 'KYCAndTransactionRisk': '14'...",https://bittrex.com/account/register?referralC...,...,Bittrex LLC\nSeafirst Fifth Avenue Plaza\n800 ...,"See a <a href=""https://bittrexglobal.zendesk.c...","Cryptocurrency\nFiat\n\nSee a <a href=""https:/...","Cryptocurrency\nFiat\n\nSee a <a href=""https:/...",False,False,"{'One': 213, 'Two': 28, 'Three': 22, 'Four': 4...",5,{'BTC': 1811.04684686804},{'BTC': 'Ƀ 1.81 K'}
BTCE,2435,/exchanges/btce/overview,/media/19560/btce.png,[Cryptocurrency],Centralized,BTCE,0,-,"{'Legal': '0', 'KYCAndTransactionRisk': '0', '...",,...,Unknown,Trading Fee: 0.2%,BTC-e code:\t0%\nBTC: 0%\t\nLTC:\t0%\nNMC: 0%\...,BTC: 0.001 BTC\nLTC:\t0.001 LTC\nNMC: 0.1 NMC\...,False,False,"{'One': 6, 'Two': 0, 'Three': 2, 'Four': 2, 'F...",6,{'BTC': 0},{'BTC': 'Ƀ 0'}
Okcoin,2436,/exchanges/okcoin/overview,/media/39838353/okcoin.png,"[Cryptocurrency, Stable Coins, Fiat]",Centralized,OKCoin,72.22,A,"{'Legal': '14.1', 'KYCAndTransactionRisk': '9....",https://www.okcoin.com/join?channelId=600006168,...,"388 Market Street, \nSuite 1300, \nSan Francis...","See a <a href=""https://support.okcoin.com/hc/e...","Cryptocurrency\nFiat\n\nSee a <a href=""https:/...","Cryptocurrency\nFiat\n\nSee a <a href=""https:/...",False,False,"{'One': 8, 'Two': 5, 'Three': 4, 'Four': 4, 'F...",7,{'BTC': 3435.03967517575},{'BTC': 'Ƀ 3.44 K'}
BTER,2437,/exchanges/bter/overview,/media/19564/bter.png,[Cryptocurrency],Centralized,BTER,0,-,"{'Legal': '0', 'KYCAndTransactionRisk': '0', '...",https://bter.com,...,,Trading Fees: 0-0.2%,Cryptocurrency\nFiat\n\nWallet\nBank Transfer,Cryptocurrency\nFiat\n\nWallet: 0-1%\nBank Tra...,False,False,"{'One': 15, 'Two': 2, 'Three': 2, 'Four': 3, '...",8,{'BTC': 0},{'BTC': 'Ƀ 0'}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Uniswap (v3),943111,/exchanges/uniswap-v3/overview,/media/38553309/uniswapv3.png,"[Cryptocurrency, Stable Coins, Tokens]",Decentralized,uniswapv3,0,-,"{'Legal': '0', 'KYCAndTransactionRisk': '0', '...",https://app.uniswap.org/#/swap,...,Unknown,Uniswap v3 offers three different fee tiers: 0...,Cryptocurrency,Cryptocurrency,False,False,"{'One': 0, 'Two': 0, 'Three': 0, 'Four': 0, 'F...",322,{'BTC': 536.629632265907},{'BTC': 'Ƀ 536.63'}
PancakeSwap (v2),943129,/exchanges/pancakeswap-v2/overview,/media/38553321/pancakeswapv2.png,"[Cryptocurrency, Stable Coins, Tokens]",Decentralized,pancakeswap,0,-,"{'Legal': '0', 'KYCAndTransactionRisk': '0', '...",https://pancakeswap.finance/,...,Unknown,When users make a token swap (trade) on the ex...,Cryptocurrencies,Cryptocurrencies,False,False,"{'One': 0, 'Two': 0, 'Three': 0, 'Four': 0, 'F...",324,{'BTC': 0},{'BTC': 'Ƀ 0'}
Crypto.com,945093,/exchanges/cryptocom/overview,/media/39198190/cryptocom.png,"[Cryptocurrency, Derivatives, Stable Coins, To...",Centralized,cryptodotcom,64.37,BB,"{'Legal': '14.5', 'KYCAndTransactionRisk': '9....",https://crypto.com/exchange,...,"Level 7, Spinola Park, Triq Mikiel Ang Borg, S...","See the full list of <a href=""https://crypto.c...","See the ful

### News
To get recent news

In [18]:
cc.get_news()

,id,guid,published_on,imageurl,title,url,source,body,tags,categories,upvotes,downvotes,lang,source_info
0,27763307,https://dailyhodl.com/?p=208869,1657058514,https://images.cryptocompare.com/news/default/...,Nexo Looking To Acquire Distressed Crypto Lend...,https://dailyhodl.com/2022/07/05/nexo-looking-...,dailyhodl,Crypto lender Nexo is announcing intentions to...,Futuremash|News|Nexo|Vauld,Market|Trading,0,0,EN,"{'name': 'The Daily Hodl', 'lang': 'EN', 'img'..."
1,27763306,https://cointelegraph.com/news/core-scientific...,1657058174,https://images.cryptocompare.com/news/default/...,Core Scientific sold $167M worth of Bitcoin ho...,https://cointelegraph.com/news/core-scientific...,cointelegraph,"The sale left the firm with ​​1,959 BTC at the...",business|bitcoin|bitcoin mining|core scientifi...,BTC|Business,0,0,EN,"{'name': 'CoinTelegraph', 'lang': 'EN', 'img':..."
2,27763295,https://seekingalpha.com/article/4521868-basic...,1657058093,https://images.cryptocompare.com/news/default/...,Basic Attention Token: The User Growth Story,https://seekingalpha.com/article/4521868-basic...,seekingalpha,BAT is the utility token of the Brave browser ...,,BAT|ICO|Altcoin|Trading|BTC,0,0,EN,"{'name': 'Seeking Alpha', 'lang': 'EN', 'img':..."
3,27763293,https://cointelegraph.com/news/bitcoin-price-s...,1657057565,https://images.cryptocompare.com/news/default/...,Bitcoin price swings 7.5% during intraday trad...,https://cointelegraph.com/news/bitcoin-price-s...,cointelegraph,"BTC bounces back to reclaim support at $20,400...",Bitcoin Price|Cryptocurrencies|Markets,BTC|Market|Trading,0,0,EN,"{'name': 'CoinTelegraph', 'lang': 'EN', 'img':..."
4,27763292,https://blockworks.co/?p=23900,1657057492,https://images.cryptocompare.com/news/default/...,Circle’s EUROC Stablecoin Will Trade Only With...,https://blockworks.co/circles-euroc-stablecoin...,blockworks,“Circle will not directly solicit customers ou...,DeFi|Regulation|Circle|Euro|EUROC|Europe|regul...,Regulation|Fiat,0,0,EN,"{'name': 'Blockworks', 'lang': 'EN', 'img': 'h..."
5,27763290,https://cointelegraph.com/news/bitcoin-exchang...,1657056743,https://images.cryptocompare.com/news/default/...,Bitcoin exchange outflows surge as 'not your k...,https://cointelegraph.com/news/bitcoin-exchang...,cointelegraph,June saw the worst monthly performance for Bit...,Bitcoin Price|Cryptocurrencies|Markets,BTC|Exchange|Market,0,0,EN,"{'name': 'CoinTelegraph', 'lang': 'EN', 'img':..."
6,27763289,https://ambcrypto.com/?p=244713,1657056618,https://images.cryptocompare.com/news/default/...,A probably win-win for Polygon [MATIC] as netw...,https://ambcrypto.com/a-probably-win-win-for-p...,ambcrypto,The Polygon [MATIC] network and its native hav...,Altcoins|News|News 1|Social|Trading View|Ether...,MATIC|Market|Altcoin|ETH|Trading,0,0,EN,"{'name': 'AMB Crypto', 'lang': 'EN', 'img': 'h..."
7,27763288,https://www.cryptoknowmics.com/news/?p=242007,1657056000,https://images.cryptocompare.com/news/default/...,CRV Technical Analysis: Buyers Plan To Escape ...,https://www.cryptoknowmics.com/news/crv-techni...,cryptoknowmics,CRV price shows a bullish trend within a symme...,Breaking News|Latest Blockchain News|Live Cryp...,Trading|CRV|Market,0,0,EN,"{'name': 'Cryptoknowmics', 'lang': 'EN', 'img'..."
8,27763287,https://cryptopotato.com/?p=201842,1657055888,https://images.cryptocompare.com/news/default/...,Institutional Interest in Shorting Bitcoin Ris...,https://cryptopotato.com/institutional-interes...,cryptopotato,Institutional investors' fixed their eyes on s...,AA News|BTCEUR|BTCGBP|BTCUSD|BTCUSDT|Crypto Ne...,BTC|Business|Market,0,0,EN,"{'name': 'Crypto Potato', 'lang': 'EN', 'img':..."
9,27763285,https://en.ethereumworldnews.com/?p=573141,1657054800,https://images.cryptocompare.com/news/default/...,"Bexplus Announcing $5,000 Rewards For New User...",https://en.ethereumworldnews.com/bexplus-annou...,ethereumworldnews,"SPONSORED: Since its creation, Bexplus has bee...",Press Releases|Sponsored Post,Business|Sponsored|Tra

To get news sources

In [19]:
cc.get_news_sources()

,name,lang,img
key,,,
cryptocompare,CryptoCompare,EN,https://images.cryptocompare.com/news/default/...
cryptoglobe,CryptoGlobe,EN,https://images.cryptocompare.com/news/default/...
theblock,TheBlock,EN,https://images.cryptocompare.com/news/default/...
cryptonewsreview,CryptoNewsReview,EN,https://images.cryptocompare.com/news/default/...
coindesk,CoinDesk,EN,https://images.cryptocompare.com/news/default/...
...,...,...,...
finbold,Finbold,EN,https://images.cryptocompare.com/news/default/...
vauld_insights,Vauld Insights,EN,https://images.cryptocompare.com/news/default/...
utoday,U.Today,EN,https://images.cryptocompare.com/news/default/...


### Base URL
To get the API's base url

In [20]:
cc.base_url

'https://min-api.cryptocompare.com/data/'

### Maximum Observations per Call
To get the maximum observations (rows) per API call

In [21]:
cc.max_obs_per_call

2000

### Rate Limit
To get the number of API calls made and remaining

In [22]:
cc.rate_limit

,calls_made,calls_left
frequency,,
second,1,49
minute,6,2494
hour,7,24993
day,7,49993
month,7,99993


## Data Request

Now that we know what data is available, we can create a data request which will be sent to the data vendor's API. 

To do this, we must first import the DataRequest class and initialize a DataRequest Object with the parameters for the data we want.

In [23]:
# import DataRequest
from cryptodatapy.extract.datarequest import DataRequest
# initialize DataRequest
data_req = DataRequest(tickers=['btc', 'eth'], fields=['close', 'active_addresses'])

We specify the desired tickers and fields in our data request. Start and end dates, as well as data frequency, are set to defaults when not specified (i.e. earliest start date, latest end date and daily frequency).

For more details on data request parameters, see `datarequest` in our documentation API reference.

## Fetch Data

Once the parameters of our data request are set, `cryptodatapy` automatically converts those parameters to the data vendor's format saving you from spending time studying each vendor's API documentation.

Each data vendor provides a range of time series data which we can access with the **fetch_datatype** methods.

To get OHLCV data, we can use the **fetch_ohlcv()** method

In [24]:
cc.fetch_ohlcv(data_req)

,,open,high,low,close,volume
date,ticker,,,,,
2010-07-17,BTC,0.04951,0.04951,0.04951,0.04951,20.00
2010-07-18,BTC,0.04951,0.08585,0.05941,0.08584,75.01
2010-07-19,BTC,0.08584,0.09307,0.07723,0.08080,574.00
2010-07-20,BTC,0.08080,0.08181,0.07426,0.07474,262.00
2010-07-21,BTC,0.07474,0.07921,0.06634,0.07921,575.00
...,...,...,...,...,...,...
2022-07-01,ETH,1069.53000,1108.86000,1032.22000,1058.01000,577588.04
2022-07-02,ETH,1058.01000,1077.18000,1027.10000,1065.69000,233775.72
2022-07-03,ETH,1065.69000,1085.63000,1040.16000,1072.95000,206730.11


To get indexes data, we can use the **fetch_indeces()** method and any avaiable index ticker(s)

In [25]:
cc.fetch_indexes(DataRequest(tickers='mvda'))

,,open,high,low,close
date,ticker,,,,
2017-07-20,MVDA,1226.16,1286.03,1217.35,1276.88
2017-07-21,MVDA,1276.88,1334.52,1166.51,1176.80
2017-07-22,MVDA,1176.80,1276.05,1169.42,1270.40
2017-07-23,MVDA,1270.40,1325.60,1193.03,1268.91
2017-07-24,MVDA,1268.91,1278.57,1225.37,1238.33
...,...,...,...,...,...
2022-07-01,MVDA,5789.86,5973.72,5594.30,5671.95
2022-07-02,MVDA,5671.95,5720.79,5591.52,5683.65
2022-07-03,MVDA,5683.65,5756.74,5576.79,5699.79


To get on-chain data, we can use the **fetch_onchain()** method

In [26]:
cc.fetch_onchain(data_req)

,,zero_balance_addresses_all_time,unique_addresses_all_time,new_addresses,active_addresses,transaction_count,transaction_count_all_time,large_transaction_count,average_transaction_value,block_height,hashrate,difficulty,block_time,block_size,current_supply
date,ticker,,,,,,,,,,,,,,
2009-01-03,BTC,0.0,1.0,1.0,1.0,1.0,1.000000e+00,0.0,50.000000,0.0,0.000000,1.000000e+00,0.000000,285.0,5.000000e+01
2009-01-04,BTC,0.0,1.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.0,5.000000e+01
2009-01-05,BTC,0.0,1.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.0,5.000000e+01
2009-01-06,BTC,0.0,1.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.0,5.000000e+01
2009-01-07,BTC,0.0,1.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000000,0.0,0.000000,0.000000e+00,0.000000,0.0,5.000000e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,ETH,144103754.0,222140102.0,58935.0,448322.0,1004306.0,1.573467e+09,4551.0,4.359225,15053225.0,862.508436,1.323852e+16,15.348854,95062.0,1.213600e+08
2022-07-01,ETH,144134515.0,222221400.0,81298.0,530163.0,1143885.0,1.574610e+09,4337.0,3.229658,15059680.0,865.319777,1.158381e+16,13.386737,80287.0,1.213735e+08
2022-07-02,ETH,144172180.0,222315668.0,94268.0,561525.0,1146133.0,1.575757e+09,2407.0,1.487246,15066168.0,867.520752,1.154751e+16,13.310930,77835.0,1.213871e+08


To get social stats data, we can use the **fetch_social()** method

In [27]:
cc.fetch_social(data_req)

,,comments,posts,followers,points,overview_page_views,analysis_page_views,markets_page_views,charts_page_views,trades_page_views,forum_page_views,...,reddit_comments_per_hour,reddit_comments_per_day,code_repo_stars,code_repo_forks,code_repo_subscribers,code_repo_open_pull_issues,code_repo_closed_pull_issues,code_repo_open_issues,code_repo_closed_issues,code_repo_contributors
date,ticker,,,,,,,,,,,,,,,,,,,,,
2017-05-26,BTC,5634.0,2370.0,5851.0,261865.0,722156.0,53665.0,73188.0,141558.0,26634.0,66635.0,...,351.56,8437.50,15817.0,9804.0,1765.0,229.0,8282.0,686.0,3537.0,NaN
2017-05-27,BTC,5827.0,2427.0,5990.0,268695.0,746321.0,54589.0,75038.0,146799.0,27456.0,69720.0,...,172.08,4130.02,15853.0,9820.0,1766.0,234.0,8283.0,688.0,3539.0,NaN
2017-05-28,BTC,5873.0,2443.0,6090.0,272055.0,762354.0,55312.0,76280.0,151348.0,27911.0,71133.0,...,138.09,3314.15,15876.0,9837.0,1766.0,234.0,8284.0,685.0,3543.0,NaN
2017-05-29,BTC,5896.0,2458.0,6199.0,275385.0,777977.0,55917.0,77388.0,154664.0,28556.0,72482.0,...,267.86,6428.57,15904.0,9845.0,1770.0,235.0,8286.0,686.0,3545.0,NaN
2017-05-30,BTC,5958.0,2471.0,6318.0,279305.0,793113.0,56560.0,78481.0,158590.0,29580.0,73833.0,...,278.21,6676.97,15932.0,9866.0,1778.0,234.0,8291.0,686.0,3546.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-01,ETH,93922.0,37508.0,90884.0,4149095.0,18913560.0,467213.0,738181.0,5648562.0,420596.0,4117886.0,...,59.64,1431.41,32217.0,11834.0,2116.0,111.0,6545.0,259.0,5588.0,629.0
2022-07-02,ETH,93934.0,37511.0,90892.0,4149490.0,18914704.0,467227.0,738185.0,5648740.0,420597.0,4117998.0,...,19.56,469.49,32217.0,11834.0,2116.0,111.0,6545.0,259.0,5588.0,629.0
2022-07-03,ETH,93936.0,37511.0,90904.0,4149810.0,18915887.0,467247.0,738195.0,5648936.0,420604.0,4118098.0,...,18.12,434.85,32217.0,11834.0,2116.0,111.0,6545.0,259.0,5588.0,629.0


Finally, to get any data type, we can use the **fetch_data()** method

In [28]:
cc.fetch_data(data_req)

AttributeError: 'CryptoCompare' object has no attribute 'get_fields'